**Modelos a correr:**
- CART (rpart)
- Modelo Logístico con Lasso (o seleccionar variables con Akaike) - GLM binomial
- Random Forest (si da)

La descriptiva se hace con toda la muestra, pero para estandarizar una variable tengo que estandarizar con la muestra de entrenamiento por un lado y con la de validación por el otro.

Sumar CV

In [ ]:
install.packages('dplyr')
install.packages('ggplot2')
install.packages('corrplot')
install.packages('GGally')
install.packages('car')
install.packages('MASS')
install.packages('stringr')
install.packages('gridExtra')
install.packages('rpart')
install.packages('rattle')
install.packages('randomForest')
install.packages('vip')
install.packages('mlbench')
install.packages('caret')
install.packages('ROCR')
install.packages('glmnet')
install.packages('ISLR')
install.packages('MKclass')
install.packages('forcats')

In [ ]:
library(dplyr)
library(ggplot2)
library(corrplot)
library(GGally)
library(car)
library(MASS)
library(stringr)
library(gridExtra)
library(rpart)
library(rattle)
library(randomForest)
library(vip)
library(mlbench)
library(caret)
library(ROCR)
library(glmnet)
library(ISLR)
library(MKclass)
library(forcats)

In [ ]:
# Cargamos el set de datos
df <- read.csv("base_preproc_4.csv", header=TRUE, sep = ';', encoding = 'latin1')
dim(df)

[1] 114  40

## Preprocesamiento de datos

In [ ]:
# renombramos columnas
colnames(df) <- c("año_de_presentacion","expediente","caratula","fecha_de_resolucion","cita_de_fallos",
  "tipo_de_actor_1", "tipo_de_actor_2", "tipo_de_actor_3", "vecinos_de_otra_provincia", "corte_causal_vecindad",
  "tipo_de_demandado_1", "tipo_de_demandado_2", "tipo_de_demandado_3", "tipo_de_demandado_4", "cantidad_de_provincias",
  "citacion_de_terceros", "provincias_citadas_terceros_tribunal", "tema", "recurso", "posterga_decision","decision_de_la_corte", "argumento",
  "involucra_recurso_interjur", "prueba_daño_interjur", "prueba_presentada", "dicta_cautelar", "audiencia_publica",
  "conformacion_mayoria", "disidencia_voto", "disidencia_sobre_total","cantidad_jueces_votan", "jueces_no_votaron", "presidente", "composicion",
  "jurisprudencia_citada", "decision_procurador", "involucra_recurso_interjur_procurador", "remision_al_procurador",
  "secretaria", "comentarios")

In [ ]:
# de fecha de resolución tomamos solo el año
df$fecha_de_resolucion <- as.integer(str_sub(df$fecha_de_resolucion,-4,-1))

# y creamos la variable tiempo de resolución como la diferencia entre ambos:
df$tiempo_de_resolucion <- df$fecha_de_resolucion - df$año_de_presentacion

# dummy si el caso tiene cita de fallos o no
df$cita_de_fallos = as.factor(ifelse(df$cita_de_fallos=='No tiene','No tiene','Tiene'))
#df$cita_de_fallos = ifelse(df$cita_de_fallos == 'No tiene', NA, df$cita_de_fallos)

# cantidad de actores
df$cantidad_de_actores_1 <- ifelse(df$tipo_de_actor_1 == "NA",0,1)
df$cantidad_de_actores_2 <- ifelse(df$tipo_de_actor_2=="NA",0,1)
df$cantidad_de_actores_3 <- ifelse(df$tipo_de_actor_3=="NA",0,1)

df$cantidad_de_actores <- rowSums(cbind(df$cantidad_de_actores_1, df$cantidad_de_actores_2, df$cantidad_de_actores_3),na.rm=TRUE)

df <- df[,!names(df) %in% c("cantidad_de_actores_1", "cantidad_de_actores_2", "cantidad_de_actores_3")]

# cantidad de demandados
df$cantidad_de_demandados_1 <- ifelse(df$tipo_de_demandado_1== "NA",0,1)
df$cantidad_de_demandados_2 <- ifelse(df$tipo_de_demandado_2=="NA",0,1)
df$cantidad_de_demandados_3 <- ifelse(df$tipo_de_demandado_3=="NA",0,1)
df$cantidad_de_demandados_4 <- ifelse(df$tipo_de_demandado_4=="NA",0,1)

df$cantidad_de_demandados <- rowSums(cbind(df$cantidad_de_demandados_1,
  df$cantidad_de_demandados_2,
  df$cantidad_de_demandados_3,
  df$cantidad_de_demandados_4),
  na.rm=TRUE)

df <- df[,!names(df) %in% c("cantidad_de_demandados_1", "cantidad_de_demandados_2", "cantidad_de_demandados_3","cantidad_de_demandados_4")]

# cantidad de provincias: pasamos NA a cero:
#df$cantidad_de_provincias = ifelse(df$cantidad_de_provincias=="NA",0,df$cantidad_de_provincias)
#df$cantidad_de_provincias = as.factor(df$cantidad_de_provincias)

df = df %>% mutate(cantidad_de_provincias = case_when(cantidad_de_provincias > 0 ~ cantidad_de_provincias, TRUE ~ 0))

# variables para los actores

# empresa
df$actor_empresa <- ifelse(tolower(df$tipo_de_actor_1) == "empresa" |
  tolower(df$tipo_de_actor_2) == "empresa" | tolower(df$tipo_de_actor_3) == "empresa",1,0)

df$actor_empresa[is.na(df$actor_empresa)] <- 0

# ente nacional / internacional
df$actor_ente_nac <- ifelse(tolower(df$tipo_de_actor_1) == "ente nacional/internacional" |
  tolower(df$tipo_de_actor_2) == "ente nacional/internacional" | tolower(df$tipo_de_actor_3) == "ente nacional/internacional",1,0)

df$actor_ente_nac[is.na(df$actor_ente_nac)] <- 0

# ente provincial
df$actor_ente_prov <- ifelse(tolower(df$tipo_de_actor_1) == "ente provincial" |
  tolower(df$tipo_de_actor_2) == "ente provincial" | tolower(df$tipo_de_actor_3) == "ente provincial",1,0)

df$actor_ente_prov[is.na(df$actor_ente_prov)] <- 0

# estado nacional
df$actor_estado_nac <- ifelse(tolower(df$tipo_de_actor_1) == "estado nacional" |
  tolower(df$tipo_de_actor_2) == "estado nacional" | tolower(df$tipo_de_actor_3) == "estado nacional",1,0)

df$actor_estado_nac[is.na(df$actor_estado_nac)] <- 0

# estado provincial
df$actor_estado_prov <- ifelse(tolower(df$tipo_de_actor_1) == "estado provincial" |
  tolower(df$tipo_de_actor_2) == "estado provincial" | tolower(df$tipo_de_actor_3) == "estado provincial",1,0)

df$actor_estado_prov[is.na(df$actor_estado_prov)] <- 0

# ministerio público fiscal de la nación
df$actor_mpfn <- ifelse(tolower(df$tipo_de_actor_1) == "ministerio público fiscal de la nación" |
  tolower(df$tipo_de_actor_2) == "ministerio público fiscal de la nación" | tolower(df$tipo_de_actor_3) == "ministerio público fiscal de la nación",1,0)

df$actor_mpfn[is.na(df$actor_mpfn)] <- 0

# municipio
df$actor_municipio <- ifelse(tolower(df$tipo_de_actor_1) == "municipio" |
  tolower(df$tipo_de_actor_2) == "municipio" | tolower(df$tipo_de_actor_3) == "municipio",1,0)

df$actor_municipio[is.na(df$actor_municipio)] <- 0

# ONG
df$actor_ong <- ifelse(tolower(df$tipo_de_actor_1) == "ong" |
  tolower(df$tipo_de_actor_2) == "ong" | tolower(df$tipo_de_actor_3) == "ong",1,0)

df$actor_ong[is.na(df$actor_ong)] <- 0

# persona física
df$actor_pers_fisica <- ifelse(tolower(df$tipo_de_actor_1) == "persona física" |
  tolower(df$tipo_de_actor_2) == "persona física" | tolower(df$tipo_de_actor_3) == "persona física",1,0)

df$actor_pers_fisica[is.na(df$actor_pers_fisica)] <- 0

# pueblos originarios
df$actor_pue_orig <- ifelse(tolower(df$tipo_de_actor_1) == "pueblos originarios" |
  tolower(df$tipo_de_actor_2) == "pueblos originarios" | tolower(df$tipo_de_actor_3) == "pueblos originarios",1,0)

df$actor_pue_orig[is.na(df$actor_pue_orig)] <- 0

# variables para los demandados
# ciudad autónoma de buenos aires
df$demandado_caba <- ifelse(df$tipo_de_demandado_1 == "Ciudad Autónoma de Buenos Aires" |
  df$tipo_de_demandado_2 == "Ciudad Autónoma de Buenos Aires" | df$tipo_de_demandado_3 == "Ciudad Autónoma de Buenos Aires" |
  df$tipo_de_demandado_4 == 'Ciudad Autónoma de Buenos Aires',1,0)

df$demandado_caba[is.na(df$demandado_caba)] <- 0

# empresas
df$demandado_empresa <- ifelse(df$tipo_de_demandado_1 == "Empresa" |
  df$tipo_de_demandado_2 == "Empresa" | df$tipo_de_demandado_3 == "Empresa" |
  df$tipo_de_demandado_4 == "Empresa",1,0)

df$demandado_empresa[is.na(df$demandado_empresa)] <- 0

# ente nacional
df$demandado_ente_nac <- ifelse(df$tipo_de_demandado_1 == "Ente nacional/internacional" |
  df$tipo_de_demandado_2 == "Ente nacional/internacional" | df$tipo_de_demandado_3 == "Ente nacional/internacional" |
  df$tipo_de_demandado_4 == "Ente nacional/internacional",1,0)

df$demandado_ente_nac[is.na(df$demandado_ente_nac)] <- 0

# ente provincial
df$demandado_ente_prov <- ifelse(df$tipo_de_demandado_1 == "Ente provincial" |
  df$tipo_de_demandado_2 == "Ente provincial" | df$tipo_de_demandado_3 == "Ente provincial" |
  df$tipo_de_demandado_4 == "Ente provincial",1,0)

df$demandado_ente_prov[is.na(df$demandado_ente_prov)] <- 0

# estado nacional
df$demandado_estado_nac <- ifelse(df$tipo_de_demandado_1 == "Estado Nacional" |
  df$tipo_de_demandado_2 == "Estado Nacional" | df$tipo_de_demandado_3 == "Estado Nacional" |
  df$tipo_de_demandado_4 == "Estado Nacional",1,0)

df$demandado_estado_nac[is.na(df$demandado_estado_nac)] <- 0

# estado provincial
df$demandado_estado_prov <- ifelse(df$tipo_de_demandado_1 == "Estado Provincial" |
  df$tipo_de_demandado_2 == "Estado Provincial" | df$tipo_de_demandado_3 == "Estado Provincial" |
  df$tipo_de_demandado_4 == "Estado Provincial",1,0)

df$demandado_estado_prov[is.na(df$demandado_estado_prov)] <- 0

# municipio
df$demandado_municipio <- ifelse(df$tipo_de_demandado_1 == "Municipio" |
  df$tipo_de_demandado_2 == "Municipio" | df$tipo_de_demandado_3 == "Municipio" |
  df$tipo_de_demandado_4 == "Municipio",1,0)

df$demandado_municipio[is.na(df$demandado_municipio)] <- 0

# organismo interjurisdiccional
df$demandado_actor_interjuris <- ifelse(df$tipo_de_demandado_1 == "Organismo Interjurisdiccional" |
  df$tipo_de_demandado_2 == "Organismo Interjurisdiccional" | df$tipo_de_demandado_3 == "Organismo Interjurisdiccional" |
  df$tipo_de_demandado_4 == "Organismo Interjurisdiccional",1,0)

df$demandado_actor_interjuris[is.na(df$demandado_actor_interjuris)] <- 0

# persona física
df$demandado_pers_fisica <- ifelse(df$tipo_de_demandado_1 == "Persona Física" |
  df$tipo_de_demandado_2 == "Persona Física" | df$tipo_de_demandado_3 == "Persona Física" |
  df$tipo_de_demandado_4 == "Persona Física",1,0)

df$demandado_pers_fisica[is.na(df$demandado_pers_fisica)] <- 0

# empresa estatal
df$demandado_empresa_estatal <- ifelse(df$tipo_de_demandado_1 == "Empresa Estatal" |
  df$tipo_de_demandado_2 == "Empresa Estatal" | df$tipo_de_demandado_3 == "Empresa Estatal" |
  df$tipo_de_demandado_4 == "Empresa Estatal",1,0)

df$demandado_empresa_estatal[is.na(df$demandado_empresa_estatal)] <- 0

# Variables anidadas:
# vecinos de otra provincia:
df = df %>%
  mutate(vecinos_de_otra_provincia_1 = as.factor(case_when(tolower(vecinos_de_otra_provincia) == "si" ~ "0",
                           vecinos_de_otra_provincia == "No" ~ "1",
                           vecinos_de_otra_provincia == "No dice" ~ "1",
                           TRUE ~ "NA")))

df = df %>%
  mutate(vecinos_de_otra_provincia_2 = as.factor(case_when(vecinos_de_otra_provincia == "Si" ~ "0",
                           vecinos_de_otra_provincia == "No" ~ "1",
                           vecinos_de_otra_provincia == "No dice" ~ "0",
                           TRUE ~ "NA")))

# citación de terceros
df = df %>%
 mutate(citacion_de_terceros_1 = as.factor(case_when(tolower(citacion_de_terceros) == "si" ~ "1",
                          tolower(citacion_de_terceros) == "no" ~ "0",
                          tolower(citacion_de_terceros) == "rechaza" ~ "1"))) # si pidieron o no citar terceros

df = df %>%
  mutate(citacion_de_terceros_2 = as.factor(case_when(tolower(citacion_de_terceros) == "si" ~ "0",
                           tolower(citacion_de_terceros) == "rechaza" ~ "1",
                           TRUE ~ "NA")))

# provincias citadas como tercero por el tribunal:
df = df %>%
  mutate(provincias_citadas_terceros_tribunal_2 = as.factor(case_when(tolower(citacion_de_terceros) == "si" ~ as.character(provincias_citadas_terceros_tribunal),
  tolower(citacion_de_terceros) == "no" ~ "NA",
  tolower(citacion_de_terceros) == "rechaza" ~ "NA")))

# tema
df$tema <- as.factor(df$tema)

# involucra recurso interjurisdiccional según la corte
df$involucra_recurso_interjur <- ifelse(df$involucra_recurso_interjur == "Remite PGN: No","No", df$involucra_recurso_interjur)

df = df %>%
  mutate(involucra_recurso_interjur = as.factor(case_when(involucra_recurso_interjur == "Si" ~ "1",
                           involucra_recurso_interjur == "No" ~ "0",
                           involucra_recurso_interjur == "No dice" ~ "NA",
                           TRUE ~ "NA")))
# df = df %>%
#   mutate(involucra_recurso_interjur_2 = as.factor(case_when(involucra_recurso_interjur == "Si" ~ "0",
#                            involucra_recurso_interjur == "No" ~ "1",
#                            involucra_recurso_interjur == "No dice" ~ "0",
#                            TRUE ~ "NA")))

df = df %>%
  mutate(prueba_daño_interjur_2 = as.factor(case_when(prueba_daño_interjur == "Argumenta que por la posición de la planta que vuelca los deshechos cloacales, el problemas es claramente interjurisdiccional" ~ "Si",
                           prueba_daño_interjur == "Remite PGN" ~ "No",
                           prueba_daño_interjur == "No" ~ "No",
                           prueba_daño_interjur == "Si" ~ "Si",
                           prueba_daño_interjur == "Insuficiente" ~ "Insuficiente",
                           prueba_daño_interjur == "No dice" ~ "No dice",
                           TRUE ~ "NA")))

# dicta cautelar: difiere que quede como NA
df = df %>%
  mutate(dicta_cautelar_2 = case_when(dicta_cautelar == "Si" ~ "1",
                           dicta_cautelar == "No" ~ "0",
                           dicta_cautelar == "Difiere" ~ "NA"))

# audiencia pública: cambiamos a binaria
df$audiencia_publica <- ifelse(tolower(df$audiencia_publica)=='si',1,0)

# conformación de la mayoría
df$conformacion_mayoria = as.factor(df$conformacion_mayoria)

# de quién es la disidencia / voto:
df = df %>%
  mutate(disidencia_argibay = case_when((conformacion_mayoria == 'Disidencia' & grepl("Argibay", disidencia_voto)) ~ 1, TRUE ~ 0))

df = df %>%
  mutate(disidencia_highton = case_when((conformacion_mayoria == 'Disidencia' & grepl("Highton", disidencia_voto)) ~ 1, TRUE ~ 0))

df = df %>%
  mutate(disidencia_lorenzetti = case_when((conformacion_mayoria == 'Disidencia' & grepl("Lorenzetti", disidencia_voto)) ~ 1, TRUE ~ 0))

df = df %>%
  mutate(voto_fayt = case_when((conformacion_mayoria == 'Voto' & grepl("Fayt", disidencia_voto)) ~ 1, TRUE ~ 0))

df = df %>%
  mutate(voto_petracchi = case_when((conformacion_mayoria == 'Voto' & grepl("Petracchi", disidencia_voto)) ~ 1, TRUE ~ 0))

df = df %>%
  mutate(voto_highton = case_when((conformacion_mayoria == 'Voto' & grepl("Highton", disidencia_voto)) ~ 1, TRUE ~ 0))

df = df %>%
  mutate(voto_argibay = case_when((conformacion_mayoria == 'Voto' & grepl("Argibay", disidencia_voto)) ~ 1, TRUE ~ 0))

df = df %>%
  mutate(voto_maqueda = case_when((conformacion_mayoria == 'Voto' & grepl("Maqueda", disidencia_voto)) ~ 1, TRUE ~ 0))

df = df %>%
  mutate(voto_zaffaroni = case_when((conformacion_mayoria == 'Voto' & grepl("Zaffaroni", disidencia_voto)) ~ 1, TRUE ~ 0))

df = df %>%
  mutate(voto_rosenkrantz = case_when((conformacion_mayoria == 'Voto' & grepl("Rosenkrantz", disidencia_voto)) ~ 1, TRUE ~ 0))

# quienes no votan:
df = df %>% mutate(no_vota_argibay = case_when(grepl("argibay", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_boggiano = case_when(grepl("boggiano", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_bossert = case_when(grepl("bossert", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_fayt = case_when(grepl("fayt", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_highton = case_when(grepl("highton", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_levene = case_when(grepl("levene", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_lopez = case_when(grepl("lopez", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_lorenzetti = case_when(grepl("lorenzetti", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_maqueda = case_when(grepl("maqueda", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_moline = case_when(grepl("moliné", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_nazareno = case_when(grepl("nazareno", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_petracchi = case_when(grepl("petracchi", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_rosatti = case_when(grepl("rosatti", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_rosenkrantz = case_when(grepl("rosenkrantz", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_vazquez = case_when(grepl("vazquez", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_zaffaroni = case_when(grepl("zaffaroni", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))

df$presidente <- factor(df$presidente)

# composición
df = df %>%
  mutate(composicion_cr1 = case_when(presidente == "CR" ~ "1",
TRUE ~ "NA"))

df = df %>%
  mutate(composicion_emo1 = case_when(presidente == "EMO" ~ "1",
TRUE ~ "NA"))

df = df %>%
  mutate(composicion_ep1 = case_when(presidente == "EP" & composicion == "EP2" ~ "1",
  presidente == "EP" & composicion != "EP2" ~ "0",
  TRUE ~ "NA"))

df = df %>%
  mutate(composicion_ep2 = case_when(presidente == "EP" & composicion == "EP3" ~ "1",
  presidente == "EP" & composicion != "EP3" ~ "0",
  TRUE ~ "NA"))

df = df %>%
  mutate(composicion_ep3 = case_when(presidente == "EP" & composicion == "EP6" ~ "1",
  presidente == "EP" & composicion != "EP6" ~ "0",
  TRUE ~ "NA"))

df = df %>%
  mutate(composicion_ep4 = case_when(presidente == "EP" & composicion == "EP7" ~ "1",
  presidente == "EP" & composicion != "EP7" ~ "0",
  TRUE ~ "NA"))

df = df %>%
  mutate(composicion_hr1 = case_when(presidente == "HR" & composicion == "HR1" ~ "1",
  presidente == "HR" & composicion != "HR1" ~ "0",
  TRUE ~ "NA"))

df = df %>%
  mutate(composicion_hr2 = case_when(presidente == "HR" & composicion == "HR2" ~ "1",
  presidente == "HR" & composicion != "HR2" ~ "0",
  TRUE ~ "NA"))

df = df %>%
  mutate(composicion_jn1 = case_when(presidente == "JN" & composicion == "JN1" ~ "1",
  presidente == "JN" & composicion != "JN1" ~ "0",
  TRUE ~ "NA"))

df = df %>%
  mutate(composicion_jn2 = case_when(presidente == "JN" & composicion == "JN3" ~ "1",
  presidente == "JN" & composicion != "JN3" ~ "0",
  TRUE ~ "NA"))

df = df %>%
  mutate(composicion_jn3 = case_when(presidente == "JN" & composicion == "JN5" ~ "1",
  presidente == "JN" & composicion != "JN5" ~ "0",
  TRUE ~ "NA"))

df = df %>%
  mutate(composicion_rl1 = case_when(presidente == "RL" & composicion == "RL1" ~ "1",
  presidente == "RL" & composicion != "RL1" ~ "0",
  TRUE ~ "NA"))

df = df %>%
  mutate(composicion_rl2 = case_when(presidente == "RL" & composicion == "RL2" ~ "1",
  presidente == "RL" & composicion != "RL2" ~ "0",
  TRUE ~ "NA"))

df = df %>%
  mutate(composicion_rl3 = case_when(presidente == "RL" & composicion == "RL3" ~ "1",
  presidente == "RL" & composicion != "RL3" ~ "0",
  TRUE ~ "NA"))

df = df %>%
  mutate(composicion_rl4 = case_when(presidente == "RL" & composicion == "RL4" ~ "1",
  presidente == "RL" & composicion != "RL4" ~ "0",
  TRUE ~ "NA"))

df = df %>%
  mutate(composicion_rl5 = case_when(presidente == "RL" & composicion == "RL7" ~ "1",
  presidente == "RL" & composicion != "RL7" ~ "0",
  TRUE ~ "NA"))

df = df %>% mutate(composicion_rl6 = case_when(presidente == "RL" & composicion == "RL7" ~ "1",
  presidente == "RL" & composicion != "RL8" ~ "0", TRUE ~ "NA"))

df = df %>%
  mutate(decision_procurador = case_when(tolower(decision_procurador) == "competencia" ~ "competencia",
                           tolower(decision_procurador) == "incompetencia" ~ "incompetencia",
                           TRUE ~ "NA"))

# involucra recurso interjurisdiccional para el procurador
df = df %>%
  mutate(involucra_recurso_interjur_procurador_1 = case_when(tolower(involucra_recurso_interjur_procurador) == "si" ~ "0",
                           tolower(involucra_recurso_interjur_procurador) == "no" ~ "1",
                           tolower(involucra_recurso_interjur_procurador) == "no dice" ~ "1",
                           TRUE ~ "NA"))
df = df %>%
  mutate(involucra_recurso_interjur_procurador_2 = case_when(tolower(involucra_recurso_interjur_procurador) == "si" ~ "0",
                           tolower(involucra_recurso_interjur_procurador) == "no" ~ "1",
                           tolower(involucra_recurso_interjur_procurador) == "no dice" ~ "0",
                           TRUE ~ "NA"))

df$remision_al_procurador <- factor(tolower(df$remision_al_procurador))

## Modelos

In [ ]:
# pasamos a dummy la decisión de la corte
#df$decision_de_la_corte = ifelse(df$decision_de_la_corte=='Rechaza',0,1)
df$decision_de_la_corte = as.factor(df$decision_de_la_corte)

In [ ]:
modelo1_vars <- c("año_de_presentacion","tiempo_de_resolucion","fecha_de_resolucion",
 "cita_de_fallos","cantidad_de_actores","actor_empresa","actor_ente_nac",
  "actor_ente_prov","actor_mpfn","actor_municipio","actor_ong","actor_estado_nac",
  "actor_estado_prov","actor_pers_fisica","actor_pue_orig","vecinos_de_otra_provincia_1","vecinos_de_otra_provincia_2",
  "cantidad_de_demandados","demandado_caba","demandado_empresa","demandado_ente_nac",
  "demandado_ente_prov","demandado_estado_nac","demandado_estado_prov","demandado_municipio",
  "demandado_actor_interjuris","demandado_pers_fisica","demandado_empresa_estatal",
  "cantidad_de_provincias","citacion_de_terceros_1","citacion_de_terceros_2","provincias_citadas_terceros_tribunal_2",
  "tema","posterga_decision","involucra_recurso_interjur",
  "prueba_daño_interjur_2","dicta_cautelar_2","audiencia_publica","conformacion_mayoria",
  "disidencia_argibay","disidencia_highton","disidencia_lorenzetti",
  "voto_fayt","voto_petracchi","voto_highton","voto_argibay","voto_maqueda",
  "voto_zaffaroni","voto_rosenkrantz",
  'no_vota_argibay','no_vota_boggiano','no_vota_bossert',
  'no_vota_fayt','no_vota_highton','no_vota_levene','no_vota_lopez','no_vota_lorenzetti',
  'no_vota_maqueda','no_vota_moline','no_vota_nazareno','no_vota_petracchi','no_vota_rosatti',
  'no_vota_rosenkrantz','no_vota_vazquez','no_vota_zaffaroni',
  'composicion_cr1','composicion_emo1','composicion_ep1','composicion_ep2',
  'composicion_ep3','composicion_ep4','composicion_hr1','composicion_hr2',
  'composicion_jn1','composicion_jn2','composicion_jn3','composicion_rl1',
  'composicion_rl2','composicion_rl3','composicion_rl4','composicion_rl5','composicion_rl6',
  'involucra_recurso_interjur_procurador_1','involucra_recurso_interjur_procurador_2',"remision_al_procurador",
  "decision_procurador","presidente",
  "decision_de_la_corte")

df_modelo1 <- df[modelo1_vars]

In [ ]:
set.seed(123)
training_sample = sample(c(TRUE, FALSE), nrow(df_modelo1), replace = T, prob = c(0.8,0.2))

train = df_modelo1[training_sample, ]
test = df_modelo1[!training_sample, ]

### Random Forest

#### Con dataset entero

In [ ]:
set.seed(7)
bag_df_entero = randomForest(decision_de_la_corte ~ ., data=df_modelo1, ntree=500, mtry=sqrt(ncol(df_modelo1)), importance = TRUE)

https://machinelearningmastery.com/tune-machine-learning-algorithms-in-r/

https://afit-r.github.io/random_forests#:~:text=mtry%20%3A%20the%20number%20of%20variables,When%20mtry%20%3Dp&text=the%20model%20equates%20to%20bagging.

In [ ]:
seed <- 7
metric <- 'Accuracy'

control <- trainControl(method="repeatedcv", number=5, repeats=3, search="grid")
set.seed(seed)
tunegrid <- expand.grid(.mtry=c(1:20))
rf_gridsearch <- train(decision_de_la_corte ~ ., data=df_modelo1, method="rf", metric=metric, tuneGrid=tunegrid, trControl=control)
# print(rf_gridsearch)
# plot(rf_gridsearch)

In [ ]:
set.seed <- 7
bag_20 = randomForest(decision_de_la_corte ~ ., data=df_modelo1, ntree=500, mtry=20, importance = TRUE)

rfImp1 <- bag_20$importance
vip(bag_20, color = 'red', fill='orange')
  # ggtitle('Random Forest Model Variable Importance')

In [ ]:
bag_20


Call:
 randomForest(formula = decision_de_la_corte ~ ., data = df_modelo1,      ntree = 500, mtry = 20, importance = TRUE) 
               Type of random forest: classification
                     Number of trees: 500
No. of variables tried at each split: 20

        OOB estimate of  error rate: 7.02%
Confusion matrix:
        Acepta Rechaza class.error
Acepta      40       5  0.11111111
Rechaza      3      66  0.04347826

In [ ]:
varImpPlot(bag_20, sort = TRUE, scale = FALSE)

In [ ]:
# test sobre datos nuevos
fit_test <- predict(bag_20, newdata = df, type='prob')

In [ ]:
perf <- prediction(fit_test[,2],df$decision_de_la_corte) # rechaza

auc = performance(perf, "auc")

pred3 = performance(perf, "tpr","fpr")

In [ ]:
cost_perf = performance(perf, "cost")

p_0 = perf@cutoffs[[1]][which.min(cost_perf@y.values[[1]])]

def.pred = rep("Acepta", length(fit_test[,2]))

def.pred[fit_test[,2] > p_0]="Rechaza"
confmat=table(df$decision_de_la_corte,def.pred,dnn = c("Real", "Predicha"))
confmat

         Predicha
Real      Acepta Rechaza
  Acepta      45       0
  Rechaza      1      68

#### Con set de entrenamiento

In [ ]:
set.seed(7)
bag_7_train = randomForest(decision_de_la_corte ~ ., data=train, ntree=500, mtry=sqrt(ncol(train)), importance = TRUE)
bag_7_train

In [ ]:
seed <- 7
metric <- 'Accuracy'

control <- trainControl(method="repeatedcv", number=5, repeats=3, search="grid")
set.seed(seed)
tunegrid <- expand.grid(.mtry=c(1:20))
rf_gridsearch <- train(decision_de_la_corte ~ ., data=train, method="rf", metric=metric, tuneGrid=tunegrid, trControl=control)
# print(rf_gridsearch)
# plot(rf_gridsearch)

In [ ]:
set.seed(7)
bag_17_train = randomForest(decision_de_la_corte ~ ., data=train, ntree=500, mtry=20, importance = TRUE)
bag_17_train

In [ ]:
rfImp1 <- bag_17_train$importance
vip(bag_17_train, color = 'red', fill='orange') +
  ggtitle('Random Forest Model Variable Importance')

In [ ]:
varImpPlot(bag_17_train, sort = TRUE, scale = FALSE)

In [ ]:
fit_test <- predict(bag_17_train, newdata = test, type='prob')

In [ ]:
perf <- prediction(fit_test[,2],test$decision_de_la_corte) # rechaza

auc = performance(perf, "auc")

pred3 = performance(perf, "tpr","fpr")

plot(pred3,col=2,lwd=2)
abline(a=0,b=1,lwd=2,lty=2,col="gray")

In [ ]:
cost_perf = performance(perf, "cost")

# https://www.listendata.com/2014/11/random-forest-with-r.html
# https://www.projectpro.io/recipes/select-best-cutoff-point-for-problem-roc-auc-curve-r

In [ ]:
p_0 = perf@cutoffs[[1]][which.min(cost_perf@y.values[[1]])]

def.pred = rep("Acepta", length(fit_test[,2]))

def.pred[fit_test[,2] > p_0]="Rechaza"
confmat=table(test$decision_de_la_corte,def.pred,dnn = c("Real", "Predicha"))
confmat

In [ ]:
df_test = df %>% filter(row_number() %in% as.numeric(rownames(test)))

In [ ]:
expedientes_erroneos <- cbind(df_test[c("expediente","decision_de_la_corte")],def.pred)
expedientes_erroneos %>% filter(decision_de_la_corte != def.pred)

Qué casos se equivoca del dataset entero

In [ ]:
fit_df_entero <- predict(bag_20, newdata = df, type='prob')

In [ ]:
perf <- prediction(fit_df_entero[,2],df$decision_de_la_corte) # rechaza

auc = performance(perf, "auc")

pred3 = performance(perf, "tpr","fpr")

plot(pred3,main="ROC Curve for Random Forest",col=2,lwd=2)
abline(a=0,b=1,lwd=2,lty=2,col="gray")

In [ ]:
cost_perf = performance(perf, "cost")

In [ ]:
p_0 = perf@cutoffs[[1]][which.min(cost_perf@y.values[[1]])]

def.pred = rep("Acepta", length(fit_df_entero[,2]))

def.pred[fit_df_entero[,2] > p_0]="Rechaza"
confmat=table(df$decision_de_la_corte,def.pred,dnn = c("Real", "Predicha"))
confmat

# ver en cuál se equivocó

In [ ]:
df_predict = df %>% filter(row_number() %in% as.numeric(rownames(df)))

In [ ]:
expedientes_erroneos_df <- cbind(df_predict[c("expediente","decision_de_la_corte")], def.pred)
expedientes_erroneos_df %>% filter(decision_de_la_corte != def.pred)

#### Con dataset entero (sin procurador)

In [ ]:
modelo1_sinproc_vars <- c("año_de_presentacion","tiempo_de_resolucion","fecha_de_resolucion",
 "cita_de_fallos","cantidad_de_actores","actor_empresa","actor_ente_nac",
  "actor_ente_prov","actor_mpfn","actor_municipio","actor_ong","actor_estado_nac",
  "actor_estado_prov","actor_pers_fisica","actor_pue_orig","vecinos_de_otra_provincia_1","vecinos_de_otra_provincia_2",
  "cantidad_de_demandados","demandado_caba","demandado_empresa","demandado_ente_nac",
  "demandado_ente_prov","demandado_estado_nac","demandado_estado_prov","demandado_municipio",
  "demandado_actor_interjuris","demandado_pers_fisica","demandado_empresa_estatal",
  "cantidad_de_provincias","citacion_de_terceros_1","citacion_de_terceros_2","provincias_citadas_terceros_tribunal_2",
  "tema","posterga_decision","involucra_recurso_interjur",
  "prueba_daño_interjur_2","dicta_cautelar_2","audiencia_publica","conformacion_mayoria",
  "disidencia_argibay","disidencia_highton","disidencia_lorenzetti",
  "voto_fayt","voto_petracchi","voto_highton","voto_argibay","voto_maqueda",
  "voto_zaffaroni","voto_rosenkrantz",
  'no_vota_argibay','no_vota_boggiano','no_vota_bossert',
  'no_vota_fayt','no_vota_highton','no_vota_levene','no_vota_lopez','no_vota_lorenzetti',
  'no_vota_maqueda','no_vota_moline','no_vota_nazareno','no_vota_petracchi','no_vota_rosatti',
  'no_vota_rosenkrantz','no_vota_vazquez','no_vota_zaffaroni',
  'composicion_cr1','composicion_emo1','composicion_ep1','composicion_ep2',
  'composicion_ep3','composicion_ep4','composicion_hr1','composicion_hr2',
  'composicion_jn1','composicion_jn2','composicion_jn3','composicion_rl1',
  'composicion_rl2','composicion_rl3','composicion_rl4','composicion_rl5','composicion_rl6',
  "decision_de_la_corte") #52

df_modelo1_sinproc <- df[modelo1_sinproc_vars]

In [ ]:
dim(df_modelo1_sinproc)

[1] 114  83

In [ ]:
set.seed(123)
training_sample_sinproc = sample(c(TRUE, FALSE), nrow(df_modelo1_sinproc), replace = T, prob = c(0.8,0.2))

train_sinproc = df_modelo1_sinproc[training_sample_sinproc, ]
test_sinproc = df_modelo1_sinproc[!training_sample_sinproc, ]

In [ ]:
seed <- 7
metric <- 'Accuracy'

control <- trainControl(method="repeatedcv", number=5, repeats=3, search="grid")
set.seed(seed)
tunegrid <- expand.grid(.mtry=c(1:20))
rf_gridsearch <- train(decision_de_la_corte ~ ., data=df_modelo1_sinproc, method="rf", metric=metric, tuneGrid=tunegrid, trControl=control)
# print(rf_gridsearch)
# plot(rf_gridsearch)

In [ ]:
set.seed(7)
bag_20 = randomForest(decision_de_la_corte ~ ., data=df_modelo1_sinproc, ntree=500, mtry=20, importance = TRUE)
bag_20

In [ ]:
rfImp1 <- bag_20$importance
vip(bag_20, color = 'red', fill='orange') +
  ggtitle('Random Forest Model Variable Importance')

In [ ]:
varImpPlot(bag_20, sort = TRUE, scale = FALSE)

In [ ]:
fit_df_entero <- predict(bag_20, newdata = df, type='prob')

In [ ]:
perf <- prediction(fit_df_entero[,2],df$decision_de_la_corte) # rechaza

auc = performance(perf, "auc")

pred3 = performance(perf, "tpr","fpr")

cost_perf = performance(perf, "cost")

In [ ]:
p_0 = perf@cutoffs[[1]][which.min(cost_perf@y.values[[1]])]

def.pred = rep("Acepta", length(fit_df_entero[,2]))

def.pred[fit_df_entero[,2] > p_0]="Rechaza"
confmat=table(df$decision_de_la_corte,def.pred,dnn = c("Real", "Predicha"))
confmat

# ver en cuál se equivocó

In [ ]:
expedientes_erroneos_df <- cbind(df[c("expediente","decision_de_la_corte")], def.pred)
expedientes_erroneos_df %>% filter(decision_de_la_corte != def.pred)

#### Con set de entrenamiento (sin Procurador)

In [ ]:
seed <- 7
metric <- 'Accuracy'

control <- trainControl(method="repeatedcv", number=5, repeats=3, search="grid")
set.seed(seed)
tunegrid <- expand.grid(.mtry=c(1:20))
rf_gridsearch <- train(decision_de_la_corte ~ ., data=train_sinproc, method="rf", metric=metric, tuneGrid=tunegrid, trControl=control)
# print(rf_gridsearch)
# plot(rf_gridsearch)

In [ ]:
set.seed(7)
bag_17_train = randomForest(decision_de_la_corte ~ ., data=train_sinproc, ntree=500, mtry=19, importance = TRUE)
bag_17_train

In [ ]:
rfImp1 <- bag_17_train$importance
vip(bag_17_train, color = 'red', fill='orange') +
  ggtitle('Random Forest Model Variable Importance')

In [ ]:
varImpPlot(bag_17_train, sort = TRUE, scale = FALSE)

In [ ]:
fit_test <- predict(bag_17_train, newdata = test_sinproc, type='prob')

In [ ]:
perf <- prediction(fit_test[,2],test_sinproc$decision_de_la_corte) # rechaza

auc = performance(perf, "auc")

pred3_sp = performance(perf, "tpr","fpr")

plot(pred3_sp,col=2,lwd=2)
abline(a=0,b=1,lwd=2,lty=2,col="gray")

In [ ]:
cost_perf = performance(perf, "cost") # averiguar qué es este costo

p_0 = perf@cutoffs[[1]][which.min(cost_perf@y.values[[1]])]

def.pred = rep("Acepta", length(fit_test[,2]))

def.pred[fit_test[,2] > p_0]="Rechaza"
confmat=table(test_sinproc$decision_de_la_corte,def.pred,dnn = c("Real", "Predicha"))
confmat

In [ ]:
df_test = df %>% filter(row_number() %in% as.numeric(rownames(test_sinproc)))

In [ ]:
expedientes_erroneos <- cbind(df_test[c("expediente","decision_de_la_corte")],def.pred)
expedientes_erroneos %>% filter(decision_de_la_corte != def.pred)

### Lasso

#### Con dataset entero

In [ ]:
glm_df_modelo1 <- glm(decision_de_la_corte ~ ., family = binomial, data=df_modelo1)

In [ ]:
#library(caret)
set.seed <- 7
x <- model.matrix(glm_df_modelo1)[,-1] # matriz de datos sin la variable b0 (la que tiene todos los unos)

ajuste.lasso = glmnet(x, df_modelo1$decision_de_la_corte, family = "binomial", alpha = 1)
plot(ajuste.lasso, label = T, xvar="lambda",lwd=2, main="Lasso Regression")

In [ ]:
set.seed <- 7
cv.lasso <- cv.glmnet(x, df_modelo1$decision_de_la_corte, alpha = 1, family = "binomial")

#Ajustamos el modelo
lasso.cv <- glmnet(x, df_modelo1$decision_de_la_corte, alpha = 1, family = "binomial",
                lambda = cv.lasso$lambda.1sd) # corremos con toda la base -> descriptivo

outcome <- coef(lasso.cv,cv.lasso$lambda.1se) # 17 variables

In [ ]:
as.matrix(outcome[outcome[,1]!=0,])

In [ ]:
fit_dfmodelo <- predict(lasso.cv, newx = x, type = 'response', s = cv.lasso$lambda.min)

# optimal cut off
optCutOff_df <- optCutoff(fit_dfmodelo, df_modelo1$decision_de_la_corte, namePos="Rechaza", perfMeasure = "YJS")

p_0_df = optCutOff_df[1]

def.pred_df = rep("Acepta", length(fit_dfmodelo))

def.pred_df[fit_dfmodelo > p_0_df]="Rechaza"
confmat_df = table(df_modelo1$decision_de_la_corte,def.pred_df,dnn = c("Real", "Predicha"))
confmat_df

In [ ]:
expedientes_erroneos_df <- cbind(df[c("expediente","decision_de_la_corte")],def.pred_df)
expedientes_erroneos_df %>% filter(decision_de_la_corte != def.pred_df)

#### Con dataset de entrenamiento

In [ ]:
x_train = x[training_sample, ]
x_test = x[!training_sample, ]

In [ ]:
ajuste.lasso_train = glmnet(x_train, train$decision_de_la_corte, family = "binomial", alpha = 1)

In [ ]:
cv.lasso_train <- cv.glmnet(x_train, train$decision_de_la_corte, alpha = 1, family = "binomial")

In [ ]:
#Ajustamos el modelo
lasso.cv_train <- glmnet(x_train, train$decision_de_la_corte, alpha = 1, family = "binomial",
                lambda = cv.lasso_train$lambda.1sd)

outcome <- coef(lasso.cv_train,cv.lasso_train$lambda.1se)

In [ ]:
as.matrix(outcome[outcome[,1]!=0,])

In [ ]:
fit_test_lasso <- predict(lasso.cv_train, newx = x_test, type = 'response', s = cv.lasso_train$lambda.min)

In [ ]:
# optimal cut off
optCutOff <- optCutoff(fit_test_lasso,test$decision_de_la_corte, namePos="Rechaza", perfMeasure = "YJS")

p_0 = optCutOff[1]

def.pred = rep("Acepta", length(fit_test_lasso))

def.pred[fit_test_lasso > p_0]="Rechaza"
confmat=table(test$decision_de_la_corte,def.pred,dnn = c("Real", "Predicha"))
confmat

In [ ]:
df_test = df %>% filter(row_number() %in% as.numeric(rownames(test)))

expedientes_erroneos <- cbind(df_test[c("expediente","decision_de_la_corte")],def.pred)
expedientes_erroneos %>% filter(decision_de_la_corte != def.pred)

In [ ]:
# perf <- prediction(fit_test[,2],test$decision_de_la_corte) # rechaza
perf_lasso <- prediction(fit_test_lasso[,1],test$decision_de_la_corte) # rechaza

auc = performance(perf_lasso, "auc")

pred3 = performance(perf_lasso, "tpr","fpr")

plot(pred3,col=2,lwd=2)
abline(a=0,b=1,lwd=2,lty=2,col="gray")

In [ ]:
auc_ROCR <- performance(perf_lasso, measure = "auc")
auc_ROCR <- auc_ROCR@y.values[[1]]
auc_ROCR

#### Con dataset entero (sin procurador)

In [ ]:
glm_df_modelo1_sinproc <- glm(decision_de_la_corte ~ ., family = binomial, data = df_modelo1_sinproc)

In [ ]:
#library(caret)
x <- model.matrix(glm_df_modelo1_sinproc)[,-1] # matriz de datos sin la variable b0 (la que tiene todos los unos)

ajuste.lasso = glmnet(x, df_modelo1_sinproc$decision_de_la_corte, family = "binomial", alpha = 1)
plot(ajuste.lasso, label = T, xvar="lambda",lwd=2, main="Lasso Regression")

In [ ]:
cv.lasso_sinproc <- cv.glmnet(x, df_modelo1_sinproc$decision_de_la_corte, alpha = 1, family = "binomial")

#Ajustamos el modelo
lasso.cv_sinproc <- glmnet(x, df_modelo1_sinproc$decision_de_la_corte, alpha = 1, family = "binomial",
                lambda = cv.lasso_sinproc$lambda.1sd) # corremos con toda la base -> descriptivo

outcome <- coef(lasso.cv_sinproc,cv.lasso_sinproc$lambda.1se) # 18 variables
# outcome

as.matrix(outcome[outcome[,1]!=0,])

In [ ]:
fit_dfmodelo_sinproc <- predict(lasso.cv_sinproc, newx = x, type = 'response', s = cv.lasso_sinproc$lambda.min)

# optimal cut off
optCutOff_df <- optCutoff(fit_dfmodelo_sinproc, df_modelo1_sinproc$decision_de_la_corte, namePos="Rechaza", perfMeasure = "YJS")

p_0_df = optCutOff_df[1]

def.pred_df = rep("Acepta", length(fit_dfmodelo_sinproc))

def.pred_df[fit_dfmodelo_sinproc > p_0_df]="Rechaza"
confmat_df = table(df_modelo1_sinproc$decision_de_la_corte,def.pred_df,dnn = c("Real", "Predicha"))
confmat_df

In [ ]:
expedientes_erroneos_df <- cbind(df[c("expediente","decision_de_la_corte")],def.pred_df)
expedientes_erroneos_df %>% filter(decision_de_la_corte != def.pred_df)

#### Con dataset de entrenamiento (sin procurador)

In [ ]:
x_train = x[training_sample, ]
x_test = x[!training_sample, ]

In [ ]:
ajuste.lasso_train = glmnet(x_train, train_sinproc$decision_de_la_corte, family = "binomial", alpha = 1)

In [ ]:
set.seed <- 7
cv.lasso_train <- cv.glmnet(x_train, train_sinproc$decision_de_la_corte, alpha = 1, family = "binomial")

#Ajustamos el modelo
lasso.cv_train <- glmnet(x_train, train_sinproc$decision_de_la_corte, alpha = 1, family = "binomial",
                lambda = cv.lasso_train$lambda.1sd)

outcome <- coef(lasso.cv_train,cv.lasso_train$lambda.1se)

as.matrix(outcome[outcome[,1]!=0,])

In [ ]:
fit_test_lasso_sp <- predict(lasso.cv_train, newx = x_test, type = 'response', s = cv.lasso_train$lambda.min)

In [ ]:
# optimal cut off
optCutOff <- optCutoff(fit_test_lasso_sp, test_sinproc$decision_de_la_corte, namePos="Rechaza", perfMeasure = "YJS")

p_0 = optCutOff[1]

def.pred = rep("Acepta", length(fit_test_lasso_sp))

def.pred[fit_test_lasso_sp > p_0]="Rechaza"
confmat=table(test_sinproc$decision_de_la_corte,def.pred,dnn = c("Real", "Predicha"))
confmat

In [ ]:
df_test = df %>% filter(row_number() %in% as.numeric(rownames(test)))

expedientes_erroneos <- cbind(df_test[c("expediente","decision_de_la_corte")],def.pred)
expedientes_erroneos %>% filter(decision_de_la_corte != def.pred)

In [ ]:
# perf <- prediction(fit_test[,2],test$decision_de_la_corte) # rechaza
perf_lasso_sp <- prediction(fit_test_lasso_sp[,1],test$decision_de_la_corte) # rechaza

auc = performance(perf_lasso_sp, "auc")

pred3 = performance(perf_lasso_sp, "tpr","fpr")

plot(pred3,col=2,lwd=2)
abline(a=0,b=1,lwd=2,lty=2,col="gray")